In [8]:
#import libraries
import pandas as pd
import lyricsgenius as genius #used to interface with Genius API

In [9]:
#token provided by Genius API
%store -r client_access_token

In [10]:
#initiate Genius
genius = genius.Genius(client_access_token)

In [11]:
def get_lyrics(track,artist):
    '''
    input:
        track(str): song's track
        artist(str): song's artist
    output: song's lyrics
    '''
    return genius.search_song(track,artist).lyrics